<a href="https://colab.research.google.com/github/hdang20/Medication_Mentioning_Tweet_Classification/blob/main/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train 10 models with BERT large to make ensemble BERT
Max_number_epochs = 4
patience =3
en_core_sci_lg, threshold = 0.7

Training dataset: Training 2020 + Validation 2020 + Training 2018

Only use prefilter for testing dataset


In [ ]:

!pip3 install scispacy
!pip3 install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
import spacy
import scispacy
import en_core_sci_lg
from scispacy.umls_linking import UmlsEntityLinker
import re
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


!pip install transformers
import torch
from transformers import AutoTokenizer,AutoModelForSequenceClassification,AdamW,BertConfig,get_linear_schedule_with_warmup
#from transformers import BertTokenizer,BertForSequenceClassification,AdamW,BertConfig,get_linear_schedule_with_warmup
import configparser,os,time,random,pickle
import numpy as np
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

learning_rate=0.00001
max_number_of_epochs=4
checkpoint_dir = "gdrive/My Drive/Colab Notebooks/bio-clinical-bert/"
gradient_clipping=1
patience=3
batch_size=16
tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
#tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
device=torch.device('cuda')

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have", "da": "the", "w/o": "without","w/": "with"}

def camel_case_split(sent):
  list_str = []
  for word in sent.split(' '):
    if word and word[0]=='#':
      regex = r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))'
      if re.search(regex, word):
        camel_case = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', word)
        for camel_case_element in camel_case:
          list_str.append(camel_case_element)
    else:
        list_str.append(word)
  sent = ' '.join(list_str)
  return sent  

def textprocess(data_path,Train=True):
  textprocess_list = []
  with open(data_path,'rt',encoding='utf8') as f:
    lines = f.readlines()
    for line in lines:
      items = line.strip().split('\t')
      items[2] = re.sub('http://\S+|https://\S+', '', items[2])
      items[2] = re.sub(' rt[ ]@[\S]+','',items[2]) #remove Retweet @[Name]
      items[2] = re.sub('@[\S]+','',items[2]) # Remove @[Name]
      items[2] = re.sub('_[\S]?','',items[2]) #remove underscore
      items[2] = re.sub('&amp;?','and',items[2]) #replace '&amp;' by 'and'
      items[2] = re.sub('&lt;','<',items[2]) #replace '&lt;' by '<'
      items[2] = re.sub('&gt;','>',items[2]) #replace '&gt;' by '>'
      items[2] = [''.join([i if ord(i) < 128 else '' for i in text]) for text in items[2]] #Remove non-ascii words or characters
      items[2] = ''.join([str(elem) for elem in items[2]])
      items[2]= re.sub('([.,!?()])', r' \1 ',items[2]) #add a space between a word and a punctuation.
      items[2] = re.sub('\s{2,}', ' ', items[2]) #add a space between a word and a punctuation.
      items[2] = camel_case_split(items[2])
      items[2] = re.sub('#','',items[2]) #remove '#' tagging
      items[2] = items[2].lower() #lowercase
      items[2] = items[2].strip() #remove whitespaces at the begining and the end of tweet
      items[2] = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in items[2].split(' ')])
      if Train:
        textprocess_list.append([items[2],items[4]])
      else:
        textprocess_list.append([items[0],items[2],items[4]])
  return textprocess_list

def mednorm(df,Train=True):
  nlp = en_core_sci_lg.load()
  linker = UmlsEntityLinker(threshold=0.7)
  nlp.add_pipe(linker)
  mednorm_list = []
  for row in df.itertuples(index=False):
    if Train:
      doc = nlp(row[0])
    else:
      doc = nlp(row[1])  
    has_umls = False
    for entity in doc.ents:
      if len(entity._.umls_ents) != 0:
        has_umls = True
        break
    if has_umls:
      if Train:
        mednorm_list.append([row[0],row[1]])
      else:  
        mednorm_list.append([row[0],row[1],row[2]])
  return mednorm_list

def preprocess(df):
  df.encoded_tokens = [tokenizer.encode_plus(text,add_special_tokens=True)['input_ids'] for text in df['tweet']] #encoded tokens for each tweet
  df.attention_mask = [tokenizer.encode_plus(text,add_special_tokens=True)['attention_mask'] for text in df['tweet']]
  labels=list(df['label'])
  encoded_tokens = list(df.encoded_tokens)
  attention_mask = list(df.attention_mask)
  return encoded_tokens,labels,attention_mask

def label_indexing(labels_train,labels_test):

    labels = set(labels_train + labels_test)
    label_to_indices,indices_to_label = {},{}

    for i,label in enumerate(sorted(labels)):
        label_to_indices[label] = i
    for key,item in label_to_indices.items():
        indices_to_label[item] = key

    encoded_labels_train = [label_to_indices[label] for label in labels_train]
    encoded_labels_test = [label_to_indices[label] for label in labels_test]

    label_size = len(label_to_indices.keys())

    return encoded_labels_train,encoded_labels_test,indices_to_label,label_size

# Convert indices to Torch tensor and dump into cuda
def feed_generator(encoded_tokens,encoded_labels,attention_mask,Training=False,output=False):

    batch_size = 16
    batch_seq = [x for x in range(int(len(encoded_tokens)/batch_size))]

    if Training:
        tmp = list(zip(encoded_tokens,encoded_labels,attention_mask))
        random.shuffle(tmp)
        shuffled_encoded_tokens,shuffled_encoded_labels,shuffled_attention_mask = zip(*tmp)
    else:
        shuffled_encoded_tokens,shuffled_encoded_labels,shuffled_attention_mask = encoded_tokens,encoded_labels,attention_mask
        if output:
            res = len(encoded_tokens)%batch_size
            if res != 0:
                batch_seq = [x for x in range(int(len(encoded_tokens)/batch_size)+1)]
            shuffled_encoded_tokens = shuffled_encoded_tokens+shuffled_encoded_tokens[:res]
            shuffled_encoded_labels = shuffled_encoded_labels+shuffled_encoded_labels[:res]
            shuffled_attention_mask = shuffled_attention_mask+shuffled_attention_mask[:res]

    for batch in batch_seq:
        maxlen_sent = max([len(i) for i in shuffled_encoded_tokens[batch*batch_size:(batch+1)*batch_size]])
        token_tensor = torch.tensor([tokens+[0]*(maxlen_sent-len(tokens)) for tokens in shuffled_encoded_tokens[batch*batch_size:(batch+1)*batch_size]])
        attention_mask = torch.tensor([tokens+[0]*(maxlen_sent-len(tokens)) for tokens in shuffled_attention_mask[batch*batch_size:(batch+1)*batch_size]]) 
        label_tensor = torch.tensor(shuffled_encoded_labels[batch*batch_size:(batch+1)*batch_size])

        token_tensor = token_tensor.to('cuda')
        label_tensor = label_tensor.to('cuda')
        attention_mask = attention_mask.to('cuda')

        yield token_tensor,label_tensor,attention_mask

def predict(model,data,output=False):
    model.eval()
    preds = []
    batch_count = 0
    for token_tensor, _, attention_mask in data:
        with torch.no_grad():
            logits = model(token_tensor,token_type_ids=None,attention_mask=attention_mask)[0]
        tmp_preds = torch.argmax(logits,-1).detach().cpu().numpy().tolist()
        preds += tmp_preds             
    return preds

def evaluate(preds,golds,indices_to_label,metric='acc'):
    if metric == 'acc':
        T,F = 0,0
        for pred,gold in zip(preds,golds):
            if pred == gold:
                T += 1
            else:
                F += 1
        metric = T/(T+F)
        print('T:{}, F:{}, F1:{:.2f}'.format(T,F,metric*100))

    elif metric == 'f1':
        TP,TN,FP,FN = 0,0,0,0
        for pred,gold in zip(preds,golds):
            if (pred=='1' or pred==1) and (gold=='1' or gold==1):
                TP += 1
            elif (pred=='1' or pred==1) and (gold!='1' and gold!=1):
                FP += 1
            elif (pred!='1' and pred!=1) and (gold=='1' or gold==1):
                FN += 1
            elif (pred!='1' and pred!=1) and (gold!='1' and gold!=1):
                TN += 1
        if TP==0: 
            prec,rec,metric = 0,0,0
        else:
            prec = TP/(TP+FP)
            rec = TP/(TP+FN)
            metric = 2*prec*rec / (prec+rec)
        print('TP:{}, TN:{}, FP:{}, FN:{}, F1:{:.2f}, Precision:{:.2f}, Recall:{:.2f}'.format(TP,TN,FP,FN,metric*100,prec*100,rec*100)) 
    return metric*100

def train(encoded_tokens_train,encoded_tokens_test,encoded_labels_train,encoded_labels_test,label_size,indices_to_label,attention_mask_train,attention_mask_test,i):
    config = BertConfig.from_pretrained('emilyalsentzer/Bio_ClinicalBERT',num_labels = label_size)
    model = AutoModelForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT',config=config)
    #config = BertConfig.from_pretrained('bert-large-uncased',num_labels = label_size)
    #model = BertForSequenceClassification.from_pretrained('bert-large-uncased',config=config)
    model.cuda()
    optimizer = AdamW(model.parameters(),lr=learning_rate)
    epochs = max_number_of_epochs
    total_steps = len(encoded_tokens_train)*epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)
    output_dir = checkpoint_dir
    f_best = 0
    patience = 0

    for epoch in range(epochs):

      model.train()
      data_train = feed_generator(encoded_tokens_train, encoded_labels_train, attention_mask_train, Training=True)
      data_test = feed_generator(encoded_tokens_test, encoded_labels_test, attention_mask_test, output=True)
            
      print("Start Epoch {} training...".format(epoch+1))
      start = time.time()
      total_loss = 0
      batch_loss = 0
      for (batch,(token_tensor,label_tensor,attention_mask)) in enumerate(data_train):
        model.zero_grad()           
        loss = model(token_tensor,token_type_ids=None,attention_mask=attention_mask,labels=label_tensor)[0]
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(),gradient_clipping)
        optimizer.step()
        scheduler.step()
        batch_loss += loss.item()
        total_loss += loss.item()
        if batch % 100 == 0 and batch != 0:
          print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss/100))
          batch_loss = 0
      print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / batch))
      print('Time taken for this epoch {} sec'.format(time.time() - start))
      
      print('\nStart evaluation...')
                    
      preds_test = predict(model,data_test)
      f_score_test = evaluate(preds_test,encoded_labels_test,indices_to_label,metric='f1')

      if f_score_test > f_best:
        patience = 0
        f_best = f_score_test
        model_to_save = model.module if hasattr(model, 'module') else model
        model_to_save.save_pretrained(output_dir +'model '+str(i)+'/')
        tokenizer.save_pretrained(output_dir +'model '+str(i)+'/')
        print('new highest score!\n'.format(patience))
      else:
        if epoch != 0:
          patience += 1
          print('no improvement for last {} epochs: highest {:.2f}\n'.format(patience,f_best))
        else:
          pass
      if patience == 3:
        print('Early stop!\n')
        break

    return(f_best)


#Training data shared-task 2018
addition_data_path = 'gdrive/My Drive/Colab Notebooks/AnnotationDRUGSInTweets_EMNLPChallenge18_TrainingSetClean.tsv'
addition_textprocess = textprocess(addition_data_path, Train=True)
addition_df = pd.DataFrame.from_records(addition_textprocess[1:])
print('addition_df:',len(addition_df))
#Training data shared-task 2020
train_data_path = 'gdrive/My Drive/Colab Notebooks/task1_training.tsv'
train_textprocess = textprocess(train_data_path, Train=True)
train_df = pd.DataFrame.from_records(train_textprocess[1:])
print('train_df:',len(train_df))
#Validation data shared-task 2020
validation_data_path = 'gdrive/My Drive/Colab Notebooks/task1_validation.tsv'
validation_textprocess = textprocess(validation_data_path, Train=True)
validation_df = pd.DataFrame.from_records(validation_textprocess[1:])
print('validation_df:',len(validation_df))
#Concatinate training and validation data of shared-task 2020 and training data of shared-task 2018, only keep one sample of duplicates
training_df = pd.concat([train_df, validation_df, addition_df])
print('training_df:',len(training_df))
training_df = training_df.drop_duplicates(keep='first')
print('training_df:',len(training_df))
training_df.columns = ['tweet','label']
print(training_df['label'].value_counts())

"""
#Prefilter train set
train_addition_mednorm = mednorm(train_addition_df,Train=True) 
train_addition_mednorm_df = pd.DataFrame.from_records(train_addition_mednorm)
print('train_addition_mednorm:',len(train_addition_mednorm_df))
train_addition_mednorm_df.columns = ['tweet','label']
print(train_addition_mednorm_df['label'].value_counts())
train_addition_medword_df=train_addition_df[train_addition_df['tweet'].str.contains("drug|med|medication")]
print('train_addition_medword:',len(train_addition_medword_df))
train_addition_medword_df.columns = ['tweet','label']
print(train_addition_medword_df['label'].value_counts())
train_addition_filter_df = pd.concat([train_addition_mednorm_df, train_addition_medword_df])
print('train_addition_filter:',len(train_addition_filter_df))
train_addition_filter_df = train_addition_filter_df.drop_duplicates(keep='first')
print('train_addition_filter:',len(train_addition_filter_df))
train_addition_filter_df.columns = ['tweet','label']
print(train_addition_filter_df['label'].value_counts())
train_addition_lelf_df=pd.concat([train_addition_df, train_addition_filter_df]).drop_duplicates(keep=False)
print('train_addition_lelf',len(train_addition_lelf_df))
train_addition_lelf_df.columns = ['tweet','label']
print(train_addition_lelf_df['label'].value_counts())

train_mednorm = mednorm(train_df,Train=True) 
train_mednorm_df = pd.DataFrame.from_records(train_mednorm)
print('train_mednorm:',len(train_mednorm_df))
train_mednorm_df.columns = ['tweet','label']
print(train_mednorm_df['label'].value_counts())
train_medword_df=train_df[train_df['tweet'].str.contains("drug|med|medication")]
print('train_medword:',len(train_medword_df))
train_medword_df.columns = ['tweet','label']
print(train_medword_df['label'].value_counts())
train_filter_df = pd.concat([train_mednorm_df, train_medword_df])
print('train_filter:',len(train_filter_df))
train_filter_df = train_filter_df.drop_duplicates(keep='first')
print('train_filter:',len(train_filter_df))
train_filter_df.columns = ['tweet','label']
print(train_filter_df['label'].value_counts())
train_lelf_df=pd.concat([train_df, train_filter_df]).drop_duplicates(keep=False)
print('train_lelf',len(train_lelf_df))
train_lelf_df.columns = ['tweet','label']
print(train_lelf_df['label'].value_counts())
"""

# Training 10 models from 10-fold cross-validation
X = np.asarray(training_df['tweet'])
y = np.asarray(training_df['label'])
skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=999)
i=1
for train_index, test_index in skf.split(X, y):
  X_train = X[train_index]
  X_test =  X[test_index]
  y_train = y[train_index]
  y_test = y[test_index]
  train_df = pd.DataFrame({'tweet': X_train,'label': y_train})
  validate_df = pd.DataFrame({'tweet': X_test,'label': y_test})
  encoded_tokens_Train,labels_Train,attention_mask_Train = preprocess(train_df)
  encoded_tokens_Test,labels_Test,attention_mask_Test = preprocess(validate_df)
  encoded_labels_Train,encoded_labels_Test,indices_to_label,label_size = label_indexing(labels_Train,labels_Test)
  train(encoded_tokens_Train,encoded_tokens_Test,encoded_labels_Train,encoded_labels_Test,label_size,indices_to_label,attention_mask_Train,attention_mask_Test,i)
  i += 1


  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
  Created wheel for en-core-sci-lg: filename=en_core_sci_lg-0.2.4-cp36-none-any.whl size=501343162 sha256=68e7ada9da9e0f761fc7b5f80ddc3ec79e2ac3608c41d24e24623508ca6bb868
  Stored in directory: /root/.cache/pip/wheels/ea/ab/e5/fa667519032799529ce6a50944a82d6ae3603819cd07836aa2
Successfully built en-core-sci-lg
Mounted at /content/gdrive



addition_df: 9622
train_df: 55419
validation_df: 13853
training_df: 78894
training_df: 72482
0    67352
1     5130
Name: label, dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:143: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:144: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

Start Epoch 1 training...
Epoch 1 Batch 100 Loss 0.3411
Epoch 1 Batch 200 Loss 0.1425
Epoch 1 Batch 300 Loss 0.1163
Epoch 1 Batch 400 Loss 0.1407
Epoch 1 Batch 500 Loss 0.0971
Epoch 1 Batch 600 Loss 0.0695
Epoch 1 Batch 700 Loss 0.0851
Epoch 1 Batch 800 Loss 0.0857
Epoch 1 Batch 900 Loss 0.0598
Epoch 1 Batch 1000 Loss 0.0680
Epoch 1 Batch 1100 Loss 0.0621
Epoch 1 Batch 1200 Loss 0.0707
Epoch 1 Batch 1300 Loss 0.0416
Epoch 1 Batch 1400 Loss 0.0981
Epoch 1 Batch 1500 Loss 0.0545
Epoch 1 Batch 1600 Loss 0.0523
Epoch 1 Batch 1700 Loss 0.0632
Epoch 1 Batch 1800 Loss 0.0635
Epoch 1 Batch 1900 Loss 0.0630
Epoch 1 Batch 2000 Loss 0.0539
Epoch 1 Batch 2100 Loss 0.0554
Epoch 1 Batch 2200 Loss 0.0551
Epoch 1 Batch 2300 Loss 0.0508
Epoch 1 Batch 2400 Loss 0.0690
Epoch 1 Batch 2500 Loss 0.0460
Epoch 1 Batch 2600 Loss 0.0602
Epoch 1 Batch 2700 Loss 0.0458
Epoch 1 Batch 2800 Loss 0.0594
Epoch 1 Batch 2900 Loss 0.0513
Epoch 1 Batch 3000 Loss 0.0528
Epoch 1 Batch 3100 Loss 0.0543
Epoch 1 Batch 3200 Los

...

Start Epoch 1 training...
Epoch 1 Batch 100 Loss 0.2617
Epoch 1 Batch 200 Loss 0.1795
Epoch 1 Batch 300 Loss 0.1140
Epoch 1 Batch 400 Loss 0.1265
Epoch 1 Batch 500 Loss 0.1174
Epoch 1 Batch 600 Loss 0.0833
Epoch 1 Batch 700 Loss 0.1043
Epoch 1 Batch 800 Loss 0.0744
Epoch 1 Batch 900 Loss 0.0573
Epoch 1 Batch 1000 Loss 0.0816
Epoch 1 Batch 1100 Loss 0.0814
Epoch 1 Batch 1200 Loss 0.0837
Epoch 1 Batch 1300 Loss 0.0841
Epoch 1 Batch 1400 Loss 0.0691
Epoch 1 Batch 1500 Loss 0.0499
Epoch 1 Batch 1600 Loss 0.0740
Epoch 1 Batch 1700 Loss 0.0388
Epoch 1 Batch 1800 Loss 0.0614
Epoch 1 Batch 1900 Loss 0.0524
Epoch 1 Batch 2000 Loss 0.0614
Epoch 1 Batch 2100 Loss 0.0682
Epoch 1 Batch 2200 Loss 0.0523
Epoch 1 Batch 2300 Loss 0.0643
Epoch 1 Batch 2400 Loss 0.0532
Epoch 1 Batch 2500 Loss 0.0656
Epoch 1 Batch 2600 Loss 0.0488
Epoch 1 Batch 2700 Loss 0.0458
Epoch 1 Batch 2800 Loss 0.0677
Epoch 1 Batch 2900 Loss 0.0506
Epoch 1 Batch 3000 Loss 0.0465
Epoch 1 Batch 3100 Loss 0.0362
Epoch 1 Batch 3200 Los

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

Start Epoch 1 training...
Epoch 1 Batch 100 Loss 0.3194
Epoch 1 Batch 200 Loss 0.1736
Epoch 1 Batch 300 Loss 0.1219
Epoch 1 Batch 400 Loss 0.1094
Epoch 1 Batch 500 Loss 0.0988
Epoch 1 Batch 600 Loss 0.1263
Epoch 1 Batch 700 Loss 0.0885
Epoch 1 Batch 800 Loss 0.0808
Epoch 1 Batch 900 Loss 0.0736
Epoch 1 Batch 1000 Loss 0.0840
Epoch 1 Batch 1100 Loss 0.0732
Epoch 1 Batch 1200 Loss 0.0679
Epoch 1 Batch 1300 Loss 0.0806
Epoch 1 Batch 1400 Loss 0.0582
Epoch 1 Batch 1500 Loss 0.0673
Epoch 1 Batch 1600 Loss 0.0600
Epoch 1 Batch 1700 Loss 0.0646
Epoch 1 Batch 1800 Loss 0.0558
Epoch 1 Batch 1900 Loss 0.0750
Epoch 1 Batch 2000 Loss 0.0547
Epoch 1 Batch 2100 Loss 0.0443
Epoch 1 Batch 2200 Loss 0.0668
Epoch 1 Batch 2300 Loss 0.0582
Epoch 1 Batch 2400 Loss 0.0619
Epoch 1 Batch 2500 Loss 0.0660
Epoch 1 Batch 2600 Loss 0.0491
Epoch 1 Batch 2700 Loss 0.0673
Epoch 1 Batch 2800 Loss 0.0442
Epoch 1 Batch 2900 Loss 0.0579
Epoch 1 Batch 3000 Loss 0.0340
Epoch 1 Batch 3100 Loss 0.0595
Epoch 1 Batch 3200 Los

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

Start Epoch 1 training...
Epoch 1 Batch 100 Loss 0.3038
Epoch 1 Batch 200 Loss 0.1832
Epoch 1 Batch 300 Loss 0.1269
Epoch 1 Batch 400 Loss 0.1020
Epoch 1 Batch 500 Loss 0.0965
Epoch 1 Batch 600 Loss 0.1096
Epoch 1 Batch 700 Loss 0.0802
Epoch 1 Batch 800 Loss 0.1206
Epoch 1 Batch 900 Loss 0.0890
Epoch 1 Batch 1000 Loss 0.0811
Epoch 1 Batch 1100 Loss 0.0480
Epoch 1 Batch 1200 Loss 0.0803
Epoch 1 Batch 1300 Loss 0.0619
Epoch 1 Batch 1400 Loss 0.0704
Epoch 1 Batch 1500 Loss 0.0634
Epoch 1 Batch 1600 Loss 0.0518
Epoch 1 Batch 1700 Loss 0.0633
Epoch 1 Batch 1800 Loss 0.0730
Epoch 1 Batch 1900 Loss 0.0538
Epoch 1 Batch 2000 Loss 0.0618
Epoch 1 Batch 2100 Loss 0.0582
Epoch 1 Batch 2200 Loss 0.0483
Epoch 1 Batch 2300 Loss 0.0787
Epoch 1 Batch 2400 Loss 0.0537
Epoch 1 Batch 2500 Loss 0.0691
Epoch 1 Batch 2600 Loss 0.0537
Epoch 1 Batch 2700 Loss 0.0499
Epoch 1 Batch 2800 Loss 0.0308
Epoch 1 Batch 2900 Loss 0.0650
Epoch 1 Batch 3000 Loss 0.0636
Epoch 1 Batch 3100 Loss 0.0384
Epoch 1 Batch 3200 Los

In [ ]:
#Test dataset shared-task 2020
test_data_path = 'test_data'
test_textprocess = textprocess(test_data_path,Train=False)
testing_df = pd.DataFrame.from_records(test_textprocess[1:])
testing_df.columns = ['tweet_id','tweet','label']
print('test_df:',len(test_df))
print(test_df['label'].value_counts())
#Pre-filter test dataset shared-task 2020
test_mednorm = mednorm(testing_df,Train=False) 
test_mednorm_df = pd.DataFrame.from_records(test_mednorm)
test_mednorm_df.columns = ['tweet_id','tweet','label']
print('test_mednorm_df:',len(test_mednorm_df))
print(test_mednorm_df['label'].value_counts())
test_medword_df=testing_df[testing_df['tweet'].str.contains("drug|med|medication")]
print('test_medword_df',len(test_medword_df))
print(test_medword_df['label'].value_counts())
test_filter_df = pd.concat([test_mednorm_df, test_medword_df]).drop_duplicates(keep='first')
print('test_filter_df',len(test_filter_df))
print(test_filter_df['label'].value_counts())
test_lelf_df=pd.concat([testing_df, test_filter_df]).drop_duplicates(keep=False)
test_lelf_df.columns = ['tweet_id','tweet','Class']
print('test_lelf_df',len(test_lelf_df))
print(test_lelf_df['Class'].value_counts())

def prob_predict(model,data,output=False):
    model.eval()
    model.cuda()
    probs = []
    batch_count = 0
    for token_tensor, _, attention_mask in data:
        with torch.no_grad():
            logits = model(token_tensor,token_type_ids=None,attention_mask=attention_mask)[0]
        tmp_preds = F.softmax(logits, -1)
        tmp_preds = tmp_preds.detach().cpu().numpy()
        one_prob = [each[1] for each in tmp_preds]
        probs +=  one_prob            
    print(len(probs))
    return probs

from transformers import BertTokenizer,BertForSequenceClassification,AdamW,BertConfig,get_linear_schedule_with_warmup
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
probability = []
for i in range(10):
  ### load pre-train data to model_1, make sure all the files are in same folder, i.e vocab , config and bin file
  model = BertForSequenceClassification.from_pretrained('gdrive/My Drive/Colab Notebooks/bert-large-v3/'+'model '+str(i+1)+'/', cache_dir=None)
  ### define def preprocess_1 and load data to def
  encoded_tokens_Test,labels_Test,attention_mask_Test = preprocess(test_filter_df)
  encoded_labels_Train,encoded_labels_Test,indices_to_label,label_size = label_indexing(labels_Test,labels_Test)
  data_test = feed_generator(encoded_tokens_Test, encoded_labels_Test, attention_mask_Test,Training=False,output=True)
  preds_test = prob_predict(model,data_test)
  probability.append(preds_test)
  i += 1

avg_probability = np.average(np.array(probability),axis=0)
#print(len(avg_probability))
avg_predicted_label = []
for prob in avg_probability:
  if prob > 0.5:
    avg_predicted_label.append(1)
  else:
    avg_predicted_label.append(0)
#print(len(avg_predicted_label))
print('Testing results of avg ensemble: ')
evaluate(avg_predicted_label,test_filter_df['label'],indices_to_label,metric='f1')


13853
0    13818
1       35
Name: label, dtype: int64
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linking_model/tfidf_vectors_sparse.npz not found in cache, downloading to /tmp/tmpwm8yil4t
Finished download, copying /tmp/tmpwm8yil4t to cache at /root/.scispacy/datasets/ea855fd121a193f03190a91417c209d4cd97e63d3ce4b456c248ef7c13a4ca77.03518aabd12de2103a27a50302f37c3d87b0f313a8be08b5ec306c9c4334b9b1.tfidf_vectors_sparse.npz
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linking_model/nmslib_index.bin not found in cache, downloading to /tmp/tmpc13416pb
Finished download, copying /tmp/tmpc13416pb to cache at /root/.scispacy/datasets/5f620d1bd549a98c005ed601a73806ea2cd1a86ae6c54bbc62bcb3b452ca2630.27a7ac6807fde6628311ff7d70b86fefc640d0eb70637b544c591722a2c16c2a.nmslib_index.bin
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linking_model/tfidf_vectorizer.joblib not found in cache, downloading to /tmp/tmpgsbp78ol
Finished download, copying /tmp/tmpgsbp78ol to cac

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/linking_model/concept_aliases.json not found in cache, downloading to /tmp/tmpjp_l3sqg
Finished download, copying /tmp/tmpjp_l3sqg to cache at /root/.scispacy/datasets/0f064d20aefab965d5772b2100f8436b3541e7d5313c76cfe5fe070902f149fe.31df9cdb04729860a81bd6c980224ed2bff582586c398d0c9b96ae4e257b9da2.concept_aliases.json
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_2017_aa_cat0129.json not found in cache, downloading to /tmp/tmpbxec9bbp
Finished download, copying /tmp/tmpbxec9bbp to cache at /root/.scispacy/datasets/13b30cd31cd37c1b52f3df6ea023061172d16e9941660e677fdbb29489af7410.4ad71d86ce780e00cab131c7e3b81acfd2f11dd80ccd61125c8bcde506f2ab8a.umls_2017_aa_cat0129.json
https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/data/umls_semantic_type_tree.tsv not found in cache, downloading to /tmp/tmpaet1cfe9
Finished download, copying /tmp/tmpaet1cfe9 to cache at /root/.scispacy/datasets/21a1012c532c3a431d60895c509f5b4d45b0f

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:210: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:211: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


9096
9096
9096
9096
9096
9096
9096
9096
9096
9096
Testing results of avg ensemble: 
TP:27, TN:9053, FP:4, FN:8, F1:81.82, Precision:87.10, Recall:77.14


81.81818181818183

In [ ]:
train_addition_lelf=pd.concat([train_addition_df,train_addition_filter_df]).drop_duplicates(keep=False)

In [ ]:
print(train_addition_df.drop_duplicates(keep='first'))

                                                  tweet label
0     these new anti-anxiety meds make me so sleepy ...     1
1     you can try vitamins for olivia ! hudson has t...     1
2     toby set up the humidifier for me in the room ...     1
3     i am an 8 . they also just started me on pitoc...     1
4     i did not take prenatals at all , people did n...     1
...                                                 ...   ...
9617  i have been up for 3 days , adderal amd redbul...     1
9618                        antibacterial sandy wipes !     0
9619  we must adopt all of them ! especially the jer...     0
9620  nuvaring is not an option for me while breast ...     1
9621          rt ibuprofen for breakfast ? runner probs     1

[60134 rows x 2 columns]


In [ ]:
test_filter_df['predict_label']=avg_predicted_label[:9092]
test_filter_df['prob']=avg_probability[:9092]
pd.set_option('display.max_colwidth', -1)
test_filter_df[test_filter_df['label']=='1'].sort_values(by=['prob'],ascending=True)[0:7]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


,tweet_id,tweet,label,predict_label,prob
4697,596665693874982913,no first thing in the morning and still have this migraine ! ! ! it is been now 4 days and nothing is working ! including a blood patch,1,0,0.011236
2749,762059592377458688,the ac can dry you out . get a saline spray . helps keep it from super drying .,1,0,0.110055
7644,407420810555445248,yeah im allergic to codiene *idunno if i spelled that right* i learned that the hard way,1,0,0.116394
6545,751262835649310722,my friend asked me where she can go to get birth control and i was telling jay about it and he goes god you sound like a mom,1,0,0.280192
5316,789840246141755392,but i got my iud just in case lmfao rosie's going to be an only child,1,0,0.457933
5277,798626411560910848,so i am just about to drink castor oil again because at this point i am tired,1,1,0.689547
5643,407566165473386496,are you good about taking ? health yourself and try out a supplement recommender to find your best bets !,1,1,0.791669


In [ ]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,AdamW,BertConfig,get_linear_schedule_with_warmup
tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
probability_1 = []
for i in range(10):
  ### load pre-train data to model_1, make sure all the files are in same folder, i.e vocab , config and bin file
  model_1 = AutoModelForSequenceClassification.from_pretrained('gdrive/My Drive/Colab Notebooks/bio-clinical-bert-v3/'+'model '+str(i+1)+'/', cache_dir=None)
  ### define def preprocess_1 and load data to def
  encoded_tokens_Test_1,labels_Test_1,attention_mask_Test_1 = preprocess(test_filter_df)
  encoded_labels_Train_1,encoded_labels_Test_1,indices_to_label_1,label_size_1 = label_indexing(labels_Test_1,labels_Test_1)
  data_test_1 = feed_generator(encoded_tokens_Test_1, encoded_labels_Test_1, attention_mask_Test_1,Training=False,output=True)
  preds_test_1 = prob_predict(model_1,data_test_1)
  probability_1.append(preds_test_1)
  i += 1

avg_probability_1 = np.average(np.array(probability_1),axis=0)

avg_predicted_label_1 = []
for prob in avg_probability_1:
  if prob > 0.5:
    avg_predicted_label_1.append(1)
  else:
    avg_predicted_label_1.append(0)
#print(len(avg_predicted_label))
print('Testing results of avg ensemble: ')
evaluate(avg_predicted_label_1,test_filter_df['label'],indices_to_label,metric='f1')


9096
9096
9096
9096
9096
9096
9096
9096
9096
9096
Testing results of avg ensemble: 
TP:26, TN:9056, FP:1, FN:9, F1:83.87, Precision:96.30, Recall:74.29


83.87096774193549

In [ ]:
test_filter_df['predict_label']=avg_predicted_label_2[:10355]
test_filter_df['prob']=avg_probability_2[:10355]
pd.set_option('display.max_colwidth', -1)
test_filter_df[test_filter_df['label']=='0'].sort_values(by=['prob'],ascending=False)[0:7]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


,tweet_id,tweet,label,predict_label,prob
3110,466014744949030912,benzino face strong as shit,0,1,0.887632
1479,879783369403510784,i was using medela bm storage bags bc you know they are medela and they are not going to leak . i ran out of them and started using the lansinoh,0,1,0.795405
0,502548414362841091,"cayden's already starting to open his eyes ! the anesthesia might wear off sooner than we thought ,",0,1,0.645316
59,799809931436118016,i wet the pad with the peri bottle and it took away the burn bc it soothed my swelling and the witch hazel pads helped too !,0,1,0.567676
2335,840809281867919361,dbn cops can teach sandton a thing or2 who still cant arrest a maxitaxi driver even with footage and all .,0,1,0.550652
3391,443166656651468800,sometimes when i am nauseated i eat a few peperoncinis and then i feel better .,0,1,0.532981
6540,879728512101429250,i struggle with my dog sleeping with me but brinyn stayed last night so it was ten times worse lol .,0,1,0.520060


In [ ]:
from transformers import BertTokenizer,BertForSequenceClassification,AdamW,BertConfig,get_linear_schedule_with_warmup
probability_2 = []
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
for i in range(10):
  ### load pre-train data to model_1, make sure all the files are in same folder, i.e vocab , config and bin file
  model_2 = BertForSequenceClassification.from_pretrained('gdrive/My Drive/Colab Notebooks/bert-base-v3/'+'model '+str(i+1)+'/', cache_dir=None)
  ### define def preprocess_1 and load data to def
  encoded_tokens_Test_2,labels_Test_2,attention_mask_Test_2 = preprocess(test_filter_df)
  encoded_labels_Train_2,encoded_labels_Test_2,indices_to_label_2,label_size_2 = label_indexing(labels_Test_2,labels_Test_2)
  data_test_2 = feed_generator(encoded_tokens_Test_2, encoded_labels_Test_2, attention_mask_Test_2,Training=False,output=True)
  preds_test_2 = prob_predict(model_2,data_test_2)
  probability_2.append(preds_test_2)
  i += 1

avg_probability_2 = np.average(np.array(probability_2),axis=0)

avg_predicted_label_2 = []
for prob in avg_probability_2:
  if prob > 0.5:
    avg_predicted_label_2.append(1)
  else:
    avg_predicted_label_2.append(0)
#print(len(avg_predicted_label))
print('Testing results of avg ensemble: ')
evaluate(avg_predicted_label_2,test_filter_df['label'],indices_to_label_2,metric='f1')


9096
9096
9096
9096
9096
9096
9096
9096
9096
9096
Testing results of avg ensemble: 
TP:26, TN:9056, FP:1, FN:9, F1:83.87, Precision:96.30, Recall:74.29


83.87096774193549

In [ ]:
avg_probability_ensemble = np.average(np.array([avg_probability,avg_probability_1,avg_probability_2]),axis=0,weights=[1/3,1/3, 1/3])

#print(len(avg_probability))
avg_predicted_ensemble_label = []
for prob in avg_probability_ensemble:
  if prob > 0.5:
    avg_predicted_ensemble_label.append(1)
  else:
    avg_predicted_ensemble_label.append(0)
#print(len(avg_predicted_label))
print('Testing results of ensemble: ')
evaluate(avg_predicted_ensemble_label[:10385],test_filter_df['label'],indices_to_label,metric='f1')

Testing results of ensemble: 
TP:26, TN:9055, FP:2, FN:9, F1:82.54, Precision:92.86, Recall:74.29


82.53968253968254

In [ ]:
#del test_lelf_df['tweet']
#del test_filter_df['tweet']
#del test_filter_df['label']
test_filter_df['Class']=avg_predicted_ensemble_label[:9092]
predict_test_df = pd.concat([test_filter_df, test_lelf_df])
#print(test_lelf_df)
#print(test_filter_df['label'].value_counts())



#predict_test_df.options.display.float_format = '{:.0f}'.format

#for i, value in enumerate(predict_test_df['tweet_id']):
#  predict_test_df['tweet_id'][i] = predict_test_df['tweet_id'][i].to_int()
predict_test_df['tweet_id'] = predict_test_df['tweet_id'].astype(str)
predict_test_df['Class'] = predict_test_df['Class'].astype(str)
test_df['tweet_id']=test_df['tweet_id'].astype(str)
del test_df['tweet']
del test_df['label']
test_df.merge(predict_test_df, left_on='tweet_id', right_on='tweet_id').to_csv('gdrive/My Drive/Colab Notebooks/prediction_task1_1.tsv',index=False,sep='\t', line_terminator = '\n')


In [ ]:
test_filter_df['predict_label']=avg_predicted_ensemble_label[:10355]
test_filter_df['prob']=avg_probability_ensemble[:10355]
pd.set_option('display.max_colwidth', -1)
test_filter_df[test_filter_df['label']=='0'].sort_values(by=['prob'],ascending=False)[:11]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


,tweet_id,tweet,label,predict_label,prob
1479,879783369403510784,i was using medela bm storage bags bc you know they are medela and they are not going to leak . i ran out of them and started using the lansinoh,0,1,0.745258
0,502548414362841091,"cayden's already starting to open his eyes ! the anesthesia might wear off sooner than we thought ,",0,1,0.725964
3110,466014744949030912,benzino face strong as shit,0,0,0.448109
9111,749930523057803265,these lil minors jus walked up on me and asked me can i buy their rellos . i do not know if i am mad that i helped condone that shit or . . .,0,0,0.376362
4801,783596946564976640,stop using echofon,0,0,0.373988
2098,625283798343766016,yeah . i took their wretched pills and syrups and did it all anyways,0,0,0.361622
8882,417361586949353472,"lucky , mine are like soft gel capsules . i heard they have gummi ones . deff getting those next lol .",0,0,0.355874
3897,660213592466268160,what sort of equip . are you using for digital illustration ? i am interested in anything but tablets ( hand/eye coordination drives me nuts ! ),0,0,0.353218
1407,831263469840302081,have you tried viva vegeria ? it is on nogalitos by the heb .,0,0,0.304541
4682,782937098093486080,"oh no , the 'make children paranoid they are being constantly spied on by putting a creepy figurine on a shelf until xmas' has begun . . .",0,0,0.292895


In [ ]:
import tensorflow
from tensorflow import summary